In [ ]:
#colab - drive libaries

import pandas as pd
from google.colab import drive
import os
drive.mount('/content/drive')

import polars as pl

In [ ]:
vol = '/content/drive/MyDrive/Dissertation/vol_surf.csv' #you should download the volatility surface with instructions from the read.me file
tickers = '/content/drive/MyDrive/Dissertation/sp500_monthly_tickers_2005_2022.csv'

In [ ]:
df_tickers = pl.read_csv(tickers)
df_tickers

In [ ]:
df_vol = df_opts = pl.read_csv(vol,try_parse_dates=True,ignore_errors= True,low_memory=False)
df_vol

In [ ]:
sp = (
    df_tickers
    .with_columns([
        pl.col("date")
          .str.strip_chars('"')
          .str.strptime(pl.Date, "%Y-%m-%d")
          .alias("month"),
        pl.col("tickers")
          .str.replace_all('"', "")
          .str.split(",")
          .alias("tickers_list"),
    ])
    .select(["month", "tickers_list"])
    .explode("tickers_list")
    .with_columns(pl.col("tickers_list").str.strip_chars().alias("ticker"))
    .select(["month", "ticker"])
    .unique()
)


In [ ]:
vol_m = (
    df_vol
    .with_columns([
        pl.col("date").dt.month_start().alias("month"),
        pl.col("ticker").str.strip_chars('"').alias("ticker") 
    ])
)

In [ ]:
vol_filtered = (
    vol_m.lazy()
        .join(sp.lazy(), on=["month", "ticker"], how="semi")
        .collect(streaming=True)
)

In [ ]:
vol_filtered

In [ ]:
vol_filtered.write_parquet('/content/drive/MyDrive/Dissertation/vol_filt_5_22.parquet',
    compression='snappy') #save the file 